<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/mistral_finetune_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting starting fine-tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

In [1]:
!nvidia-smi

Thu Jun  6 01:53:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

## Installation

Clone the `mistral-finetune` repo:


In [2]:
%cd /content/
!git clone https://github.com/mistralai/mistral-finetune.git

/content
Cloning into 'mistral-finetune'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 194 (delta 35), reused 15 (delta 15), pack-reused 139
Receiving objects: 100% (194/194), 147.06 KiB | 16.34 MiB/s, done.
Resolving deltas: 100% (88/88), done.


Install all required dependencies:

In [ ]:
!pip install -r /content/mistral-finetune/requirements.txt -q

## Model download

In [ ]:
!wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

In [5]:
!DIR=/content/mistral_models && mkdir -p $DIR && tar -xf mistral-7B-v0.3.tar -C $DIR

In [6]:
# Alternatively, you can download the model from Hugging Face

# !pip install huggingface_hub
# from huggingface_hub import snapshot_download
# from pathlib import Path

# mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
# mistral_models_path.mkdir(parents=True, exist_ok=True)

# snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

#! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
#! rm -r /root/mistral_models/7B-v0.3

In [7]:
!ls /content/mistral_models

consolidated.safetensors  params.json  tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [8]:
%cd /content/

/content


In [9]:
# make a new directory called data
!mkdir -p data

In [10]:
# navigate to this data directory
%cd /content/data

/content/data


In [11]:
# read data into a pandas dataframe
import pandas as pd

df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

In [12]:
# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [13]:
# save data into .jsonl files
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [14]:
!ls /content/data

ultrachat_chunk_eval.jsonl  ultrachat_chunk_train.jsonl


In [15]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


In [16]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that doesn't have the right format:

!python -m utils.reformat_data /content/data/ultrachat_chunk_train.jsonl

Skip 3674th sample
Skip 9176th sample
Skip 10559th sample
Skip 13293th sample
Skip 13973th sample
Skip 15219th sample


In [17]:
# eval data looks all good
!python -m utils.reformat_data /content/data/ultrachat_chunk_eval.jsonl

In [25]:
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.2/289.2 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00


In [ ]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


## Start training

In [19]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [20]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/data/ultrachat_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/ultrachat_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence lentgh of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 8192
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/test_ultra"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [21]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

In [22]:
# start training

!torchrun --nproc-per-node 1 -m train example.yaml

2024-06-06 02:05:41.061251: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 02:05:41.112747: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 02:05:41.112807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 02:05:41.114379: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-06 02:05:41.122158: I tensorflow/core/platform/cpu_feature_guar

## Inference

In [23]:
!pip install mistral_inference -q

In [27]:
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/mistral_models/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("/content/mistral_models")  # change to extracted model dir
model.load_lora("/content/test_ultra/checkpoints/checkpoint_000100/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=512, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

Machine learning is a subset of artificial intelligence that involves the use of algorithms to learn from data and make predictions or decisions without being explicitly programmed. It is a type of data analysis that allows computers to learn from experience and improve their performance over time. Machine learning algorithms can be trained on large datasets to identify patterns and make predictions based on those patterns. The goal of machine learning is to enable computers to perform tasks that would otherwise require human intelligence, such as recognizing images, understanding natural language, and making predictions about future events.
